In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [2]:
#export
import sys
from os.path import join

sys.path.insert(0, '/'.join(sys.path[0].split('/')[:-1] + ['scripts']))
from training import *

In [3]:
#export
import re

def camel2snake(name):
    camel_re1 = re.compile('(.)([A-Z][a-z]+)')
    camel_re2 = re.compile('([a-z0-9])([A-Z])')
    s1 = re.sub(camel_re1, r'\1_\2', name)
    return re.sub(camel_re2, r'\1_\2', s1).lower()

class Callback():
    order = 0
    def __getattr__(self, k): 
        # delegate attribute checking to runner
        return getattr(self.runner, k)

    def set_runner(self, runner):
        self.runner = runner
        
    @property
    def name(self):
        return re.sub(r'Callback$', '', self.__class__.__name__) or 'callback'
        
    def __repr__(self):
        return f'(callback) {camel2snake(self.name)}'
    
    def __call__(self, cb_name):
        fn = getattr(self, cb_name, None)
        if fn and fn(): 
            return True
        return False

In [4]:
camel2snake('helloWorldMyNameIsJack')

'hello_world_my_name_is_jack'

In [5]:
#export
class TrainEval(Callback):
    def before_train(self):
        self.model.train()

    def before_valid(self):
        self.model.eval_()

class ItersStop(Callback):
    def after_batch(self):
        print(f'iteration: {self.iters_count}')
        if self.iters_count > 10:
            self.runner.stop = True

class AvgStats():
    def __init__(self, metrics, training): 
        self.metrics = metrics
        self.training = training
    
    def reset(self):
        self.count = 0
        self.total_loss = torch.Tensor([0])
        self.totals = [torch.Tensor([0])] * len(self.metrics)
        
    @property
    def all_stats(self): return [self.total_loss] + self.totals
    
    @property
    def avg_stats(self): return [s.item()/self.count for s in self.all_stats]
    
    def __repr__(self):
        if not self.count: 
            return ''
        return f"{'train' if self.training else 'valid'} metrics - {self.avg_stats}"

    def accumulate(self, runner):
        batch_size = runner.x_batch.shape[0]
        self.count += batch_size
        self.total_loss = runner.loss * batch_size
        for i, metric in enumerate(self.metrics):
            self.totals[i] += metric(runner.pred, runner.y_batch) * batch_size

class StatsLogging(Callback):
    def __init__(self, metrics):
        self.train_stats = AvgStats(metrics, True)
        self.valid_stats = AvgStats(metrics, False)
        
    def before_epoch(self):
        self.train_stats.reset()
        self.valid_stats.reset()
        
    def after_loss(self):
        stats = self.train_stats if self.model.training else self.valid_stats
        stats.accumulate(self.runner)
    
    def after_epoch(self):
        print(f'Epoch - {self.epoch}\n{self.train_stats}\n{self.valid_stats}\n')

In [6]:
#export
class Runner():
    def __init__(self, learner, callbacks=[]):
        self.learner = learner
        self.stop = False
        self.callbacks = sorted([TrainEval()] + callbacks, key=lambda cb: cb.order)
        for callback in self.callbacks:
            callback.runner = self
    
    def __repr__(self):
        return f'{self.learner}\n(Callbacks) ' + ' '.join(map(lambda cb: cb.name, self.callbacks))
        
    @property
    def data_bunch(self): return self.learner.data_bunch
    @property
    def model(self):      return self.learner.model
    @property
    def loss_fn(self):    return self.learner.loss_fn
    @property
    def optimizer(self):  return self.learner.optimizer

    def one_batch(self, x_batch, y_batch):
        self.x_batch = x_batch
        self.y_batch = y_batch
        
        if self('before_batch'):     return
        self.pred = self.model(self.x_batch)
        if self('after_pred'):       return
        self.loss = self.loss_fn(self.pred, self.y_batch)
        if self('after_loss'):       return
        if not self.model.training:  return
        self.loss_fn.backward()
        if self('after_loss_back'):  return
        self.model.backward()
        if self('after_model_back'): return
        self.optimizer.step()
        if self('after_step'):       return
        self.optimizer.zero_grad()

    def all_batches(self):
        data_loader = self.data_bunch.train_dl if self.model.training else self.data_bunch.valid_dl
        self.iters_count, self.iters = 0, len(data_loader)
        for x_batch, y_batch in data_loader:
            if self.stop: break
            self.one_batch(x_batch, y_batch)
            self.iters_count += 1
            self('after_batch')

    def fit(self, num_epochs):
        self.num_epochs = num_epochs

        for callback in self.callbacks:
            callback.set_runner(self)
            
        if self('before_fit'):       return
        for epoch in range(1, num_epochs+1):
            if self.stop: break
            self.epoch = epoch
            if self('before_epoch'): return
            if self('before_train'): return 
            self.all_batches()
            if self('before_valid'): return
            self.all_batches()
            if self('after_epoch'): break
        self('after_fit')

    def __call__(self, callback_name):
        for callback in self.callbacks:
            if callback(callback_name):
                return True
        return False

In [7]:
num_hidden = 50
batch_size = 64
num_epochs = 20
learning_rate = 0.1

data_bunch = get_data_bunch(*get_mnist_data(), batch_size)
model, optimizer = get_model(data_bunch, learning_rate, num_hidden)
loss_fn = CrossEntropy()
learner = Learner(model, optimizer, loss_fn, data_bunch)

In [8]:
runner = Runner(learner, [StatsLogging([compute_accuracy])])
print(runner)

(DataBunch) 
	(DataLoader) 
		(Dataset) x: (50000, 784), y: (50000,)
		(Sampler) total: 50000, batch_size: 64, shuffle: True
	(DataLoader) 
		(Dataset) x: (10000, 784), y: (10000,)
		(Sampler) total: 10000, batch_size: 128, shuffle: False
(Sequential)
	(Layer1) Linear(784, 50)
	(Layer2) ReLU()
	(Layer3) Linear(50, 10)
(CrossEntropy)
(Optimizer) num_params: 4, learning_rate: 0.1
(Callbacks) TrainEval StatsLogging


In [9]:
runner.fit(5)

Epoch - 1
train metrics - [5.21607780456543e-05, 0.88386]
valid metrics - [0.00013459482192993165, 0.9249]

Epoch - 2
train metrics - [8.116597175598144e-05, 0.92818]
valid metrics - [9.224777221679688e-05, 0.9399]

Epoch - 3
train metrics - [5.448760986328125e-06, 0.94276]
valid metrics - [7.916407585144043e-05, 0.953]

Epoch - 4
train metrics - [1.526219367980957e-05, 0.95276]
valid metrics - [7.73298740386963e-05, 0.9573]

Epoch - 5
train metrics - [4.177288055419922e-05, 0.95864]
valid metrics - [7.447586059570312e-05, 0.9617]

